In [1]:
import os
import lightning as L
from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import RichProgressBar

import models
import config_tools
import dataset

In [2]:
config = config_tools.load_config('configs/faster_rcnn_config.toml')

In [3]:
if os.path.exists(config['metadata_parameters']['path_to_train_metadata']) is False:
    dataset.AtomicDatasetV2.create_train_test_split(config)

In [4]:
train_dataset = dataset.AtomicDatasetV2('/workspace/code/atomic-hack-2.0-solution/metadata/train.csv', config['metadata_parameters']['path_to_data'], config)
test_dataset = dataset.AtomicDatasetV2('/workspace/code/atomic-hack-2.0-solution/metadata/test.csv', config['metadata_parameters']['path_to_data'], config)

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataloader = DataLoader(
    train_dataset, 
    shuffle=True, 
    batch_size=config['model_parameters']['batch_size'], 
    num_workers=3, 
    pin_memory=True,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    test_dataset, 
    shuffle=False, 
    batch_size=config['model_parameters']['batch_size'], 
    num_workers=3, 
    pin_memory=True,
    collate_fn=collate_fn
)

In [5]:
model = models.AtomicFasterRCNN(config, 5)

/workspace/python_envs/hack/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/workspace/python_envs/hack/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
trainer = L.Trainer(
    max_epochs=config['model_parameters']['epochs'], 
    gradient_clip_val=config['model_parameters']['grad_clipping'],
    callbacks=[RichProgressBar()],
    accumulate_grad_batches=64
)

trainer.fit(model, train_dataloader)

Epoch 0/29 ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27/600 0:00:32 • 0:10:58 0.87it/s v_num: 83.000 train_loss: 0.079